In [0]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import random
import time

In [0]:
epochs_num=100
batch_sz=32

In [0]:
cifar10 = tf.keras.datasets.cifar10
(x_train, _), (x_test, _) = cifar10.load_data()

x_train, x_test = x_train/255.0, x_test/255.0
y_train, y_test = x_train, x_test

In [0]:
from google.colab import drive
drive.mount('./MyDrive')

In [0]:
cd MyDrive/My Drive

In [0]:
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255.0
   np_image = np.expand_dims(np_image, axis=0)
   print(np_image.shape)
   return np_image

img = load('./noisy.png')

In [0]:
print(np.shape(x_train))
print(np.shape(x_test))
print(np.shape(y_train))
print(np.shape(y_test))

In [0]:
#model2
with tf.device('/device:GPU:0'):
    inputs = layers.Input(shape=(None,None,3))
    input_with_gaussian = layers.GaussianNoise(0.1)(inputs)

    layer1 = layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='relu')(input_with_gaussian)  
    
    layer2 = layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='relu')(layer1)  
    
    layer3 = layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='relu')(layer2)  
    
    layer4 = layers.Conv2D(filters=64, kernel_size=3, padding='same',activation='relu')(layer3)  
    
    layer5 = layers.Conv2D(filters=3, kernel_size=3, padding='same')(layer4)  
    
    predictions = layers.Add()([layer5, input_with_gaussian])
    
    model = Model(inputs=inputs, outputs=predictions)

    model.compile(optimizer='adam', loss='mean_squared_error',
              metrics=['mean_squared_error'])
    
    model.fit(x_train, y_train, epochs=epochs_num, batch_size=batch_sz)
    
    model.evaluate(x_test, y_test, verbose=2)

    result = model.predict(img)
    

In [0]:
result

In [0]:
result = result.reshape(512,512,3)
result *= 255.0
result = Image.fromarray(result.astype(np.uint8), 'RGB')
result.save('Model2.png')
result